$A = \mathbb{F}_3A_5$. 

$A = B_0 \oplus B_1 = \left(P(\chi_1) \oplus P(\chi_4)^{\oplus 4}\right) \oplus P(\chi_2 \oplus \chi_3)^{\oplus 3}$.

We have an adhoc way to find idempotents for indecomposable projectives $P(\chi_1)$ and $P(\chi_4)$. Namely, using idempotents associated to irreps of subgroups (e.g., $\displaystyle \frac{1}{4} \sum_{k \in K_4} k \in \mathbb{F}_3K_4 \subset \mathbb{F}_3A_5$).

These idempotents can be projected to the principal block using the associated central idempotent $CI_0$.

Then, letting $e$ denote such an idempotent, primitive idempotents lie in the subalgebra $e(\mathbb{F}_3A_5)e$. It many cases this is small (e.g., for the $K_4$ trivial repn example, this algebra is $6$-diml). A brute force check of all possible elements produces idempotents.

The cell below performs such a construction (there are minimal comments is this was done in depth in a previous notebook).

In [1]:
def dimAe(e, two_sided = False):
    """
    Given idempotent e, find dimension of F_3A_5e if two_sided = False, and eF_3A_5e if two_sided = True
    """
    if two_sided:
        spanning_set = [e * g * e for g in A5]
    else:
        spanning_set = [g * e for g in A5]

    #create coefficient matrix for each element, the rank of this matrix is the rank/dimension.
    coeff_array = []
    for ele in spanning_set:
        coeffs = ele.monomial_coefficients() #dict of form{'<grp_element>': <coeff>}
        coeff_list = []
        for g in A5:
            try:
                #if g has non-zero coefficient, coeffs[g] will return it, else it has 0 coefficient
                coeff_list.append(coeffs[g])
            except:
                coeff_list.append(0)
        coeff_array.append(coeff_list)
    coeff_matrix = Matrix(GF(3), coeff_array)
    rank_of_matrix = coeff_matrix.rank()
    return rank_of_matrix

def coeffs_of_basis_two_sided(e):
    """
    Given idempotent e, finds coefficients of basis of eAe, where A = F_3A_5
    (ordering of coeffs follows order of [g for g in AlternatingGroup(5)])
    """
    spanning_set = [e * g * e for g in A5]
    coeff_array = []
    for ele in spanning_set:
        coeffs = ele.monomial_coefficients() #dict of form{'<grp_element>': <coeff>}
        coeff_list = []
        for g in A5:
            try:
                #if g has non-zero coefficient, coeffs[g] will return it, else it has 0 coefficient
                coeff_list.append(coeffs[g])
            except:
                coeff_list.append(0)
        coeff_array.append(coeff_list)
    coeff_matrix = Matrix(GF(3), coeff_array)
    
    #Now, just apply RREF, and extract the non zero rows (rows with a pivot)
    return [v for v in coeff_matrix.rref().rows() if v.is_zero() == False]

def find_idempotents(basis):
    possible_coeffs = cartesian_product([[0,1,2]] * 6)

    idempotents_list = []

    for coeffs in possible_coeffs:
        test_element = sum(coeff * ele for (coeff,ele) in zip(coeffs, basis))
        if test_element**2 == test_element:
            idempotents_list.append(test_element)
    return idempotents_list

A5 = AlternatingGroup(5)
F3 = GF(3)
A = GroupAlgebra(A5, F3)

#Central idempotent for block B1 is given by conjugacy class sums on character values of \rho_2 + \rho_3 = (6,-2,0,1,1) for classes 1,2,3,5A,5B respectively
CI_1_char_values = [6,-2,0,1,1]
conj_classes_A5 = A5.conjugacy_classes() # these classes are in the order 1,2,3,5,5, so order is compatible with above

CI_1 = 2*sum(val * A(ele) for (val, ccl) in zip(CI_1_char_values,conj_classes_A5) for ele in ccl) #Note: 2 = 1/20 in F_3
assert CI_1**2 == CI_1
CI_0 = 1 - CI_1
assert CI_0**2 == CI_0

g1 = A5("(1,2)(3,4)")
g2 = A5("(1,3)(2,4)")
H = A5.subgroup([g1, g2])

xbar = 1*sum(A(ele) for ele in H) #xbar is reduction modulo 3 (of "trivial idempotent"). Note that 1 = 1/4 = 1/|H| in F_3
assert xbar**2 == xbar

assert CI_0 * xbar == xbar #projection of xbar to principal block gives xbar. That is, xbar is already in the principal block

xAx_basis_coeffs = coeffs_of_basis_two_sided(xbar)
xAx_basis = [sum(coeff * A(g) for (coeff,g) in zip(coeffs,A5)) for coeffs in xAx_basis_coeffs]

idempotents_in_xAx = find_idempotents(xAx_basis)

#Of these idempotents, e, dim(Ae) = 0,6,9,15. Find an idempotent for dim(Ae) = 6 and 9.

idem_P1 = None
idem_P4 = None

for idem in idempotents_in_xAx:
    if (not idem_P1) and dimAe(idem) == 6:
       idem_P1 = idem
    if (not idem_P4) and dimAe(idem) == 9:
       idem_P4 = idem
    if idem_P1 and idem_P4:
        break

assert idem_P1**2 == idem_P1
assert dimAe(idem_P1) == 6

assert idem_P4**2 == idem_P4
assert dimAe(idem_P4) == 9

print(idem_P1)
print("")
print(idem_P4)

() + 2*(2,3)(4,5) + 2*(2,3,4) + 2*(2,3,5) + 2*(2,4,3) + 2*(2,5,3) + (1,2)(3,4) + 2*(1,2,3) + 2*(1,2,4,5,3) + 2*(1,2,4) + 2*(1,2,4,3,5) + 2*(1,2,5,4,3) + 2*(1,3,2) + 2*(1,3,4,5,2) + 2*(1,3,5,4,2) + 2*(1,3,4) + (1,3)(2,4) + 2*(1,3,4,2,5) + 2*(1,4,2) + 2*(1,4,3) + 2*(1,4,5) + 2*(1,4)(3,5) + (1,4)(2,3) + 2*(1,4)(2,5) + 2*(1,5,3,4,2) + 2*(1,5,4) + 2*(1,5)(2,3) + 2*(1,5,2,4,3)

(2,3)(4,5) + (2,3,4) + (2,3,5) + (2,4,3) + (2,5,3) + (1,2,3) + (1,2,4,5,3) + (1,2,4) + (1,2,4,3,5) + (1,2,5,4,3) + (1,3,2) + (1,3,4,5,2) + (1,3,5,4,2) + (1,3,4) + (1,3,4,2,5) + (1,4,2) + (1,4,3) + (1,4,5) + (1,4)(3,5) + (1,4)(2,5) + (1,5,3,4,2) + (1,5,4) + (1,5)(2,3) + (1,5,2,4,3)


# Lifting

To lift, we lift to 3-adic numbers, and then check if in fact our number is rational. Similar to the reals, a 3-adic number is rational if it has repeating digits.

## Lifting from $\mathbb{F}_3A_5$ to $\hat{\mathbb{Z}_3}A_5$ (but hopefully $\mathbb{Q}A_5$)

As $\hat{\mathbb{Z}_3}$ is complete, it is always possible to perform such a lift. 

In practice, we lift to $\mathbb{Z}/3^c\mathbb{Z}$, which will give us the first $c$ coefficients of the lift.

As we are hoping for the coefficients to repeat  (since this is required for our $3$-adic number to be rational), we hope to spot a repeating pattern in the first $c$ coefficients.

Then, assuming the repeating pattern holds, we calculate the (concrete?) rational coefficients and check idempotency. 

## The iterative process.

- We start with any lift from $\mathbb{F}_3A_5$ to $\mathbb{Z}_{(3)}A_5 \subset \mathbb{Q}A_5$. The naive lift induced from set map $\mathbb{F}_3 \rightarrow \mathbb{Z}$ by "removing the hat" will do. 
- Denote this initial lift by $e_0$. Then $e_0 = \bar{e} (\text{ mod } 3)$, but $e_0$ is certainly not idempotent.
- Iteratively define $e_{k+1} = 3e_k^2 - 2e_k^3$. 
    - We have $e_{k+1}\equiv 3e_k^2 - 2e_k^3 \equiv e_k^3  (\text{ mod } 3) $. So if $e_k \equiv \bar{e} (\text{ mod } 3)$, then $e_{k+1}$ will be too. So all the $e_i$ are lifts of $\bar{e}$.
    - Let $a_k = e_k^2 - e_k$. This is the error term (a measure of how far $e_k$ is for being idempotent).
    - Claim: If $a_k \equiv 0 (\text{ mod } 3^n)$ then $a_{k+1} \equiv 0 (\text{ mod } 3^{2n})$. The proof is to show that $a_{k+1} = a_k^2(\text{stuff})$.
    - Thus, $a_0 \equiv 0 (\text{ mod } 3^1)$, $a_1 \equiv 0 (\text{ mod } 3^2)$, $a_2 \equiv 0 (\text{ mod } 3^{2^2} = 3^4)$, $a_3 \equiv 0 (\text{ mod } 3^{2^3} = 3^8)$, etc
- If we work in the ring $\mathbb{Z}/3^c\mathbb{Z}$, then this iteration stabilises whenever $K$ is such that $2^K > c$. 
- This will find the first $c$ $3$-adic coefficients of the lift.

In [2]:
def refine_lift(initial_lift, num_coeffs):
    c = num_coeffs
    N = 3**c
    RA = GroupAlgebra(A5,IntegerModRing(N))
    old_lift = None
    current_lift = sum(int(coeff)*RA(ele) for (ele,coeff) in initial_lift.monomial_coefficients().items())
    while current_lift != old_lift:
        old_lift = current_lift
        current_lift = 3*current_lift**2 - 2*current_lift**3
    return current_lift

def int_coeffs_to_3adic(ele_coeff_dict,num_coeffs):
    """
    Takes a dictionary with entries <str>:<int> and computes the first `num_coeffs` terms of 3-adic expansion of <int> (this is just base 3 number)
    """
    def int_to_base3(n):
        n %= 3**num_coeffs #our ints are only accurate to `num_coeffs` places, so the larger values are nonense
        expansion = ''
        for i in range(num_coeffs):#doing it this way rather than checking n==0 allows left hand side entries to be padded with 0s.
            expansion = str(n%3)+expansion
            n = n // 3
        return '...'+expansion
    expansion_dict = dict()
    for key,val in ele_coeff_dict.items():
        expansion_dict[key] = int_to_base3(int(val))
    return expansion_dict

# 3-adic coefficients to rational coefficients

Foreshadowng: The coefficients for the refined lift of the projective cover $P(\chi_1)$ and $P(\chi_4)$ appear to have repeating digits (print the dataframe below to see this).

Namely all coefficients are of the form $(abcd)ef$, where $( - )$ denotes repeating digits (leftwards). To produce a rational number from such a $3$-adic number we do the following:

$x = (abcd)abcdef$

$81x = (abcd)ef0000$ (everything shifts 4 places to the left, since $81 = 3^4$).

$\implies 80x = 3^5e+3^4f-(3^5a+3^4b+3^3c+3^2d+3e+f)$

$\implies x = \frac{1}{80}\big(3^5e+3^4f-(3^5a+3^4b+3^3c+3^2d+3e+f)\big)$

In [3]:
def rational_from_3_adic(expansion:str):
    """
    Takes a 3-adic expansion (passed as a string) of the form (abcd)ef and returns the fraction it represents.
    """
    a,b,c,d,e,f = map(int,expansion[-6:])
    numerator = 3**5*e+3**4*f-(3**5*a+3**4*b+3**3*c+3**2*d+3*e+f)
    return numerator/Integer(80)

In [4]:
###############
# For idem_P1 #
###############

import pandas as pd


QA = GroupAlgebra(A5,QQ)
num_coeffs_of_accuracy_in_lift_refinement = 40


initial_lift_of_idem_P1 = sum(int(coeff)*QA(g) for (g,coeff) in idem_P1.monomial_coefficients().items())
refined_lift_of_idem_P1 = refine_lift(initial_lift_of_idem_P1, num_coeffs_of_accuracy_in_lift_refinement)
refined_lift_of_idem_P1_3_adic_coefficients = int_coeffs_to_3adic(refined_lift_of_idem_P1.monomial_coefficients(), num_coeffs_of_accuracy_in_lift_refinement)

idem_P1_lift_df = pd.DataFrame.from_dict(refined_lift_of_idem_P1_3_adic_coefficients, orient='index',columns=['3-adic coefficients'])
idem_P1_lift_df['rational_coefficients'] = idem_P1_lift_df['3-adic coefficients'].apply(rational_from_3_adic)
idem_P1_lift_df.index.name = "A5 element"

rational_lift_P1 = (idem_P1_lift_df.index.to_series().apply(lambda x: QA(x)) * idem_P1_lift_df['rational_coefficients']).sum()

#is it idempotent?
assert rational_lift_P1**2 == rational_lift_P1
#does it reduce modulo 3 to idem_P1?
assert sum(GF(3)(coeff)*A(ele) for (ele,coeff) in rational_lift_P1.monomial_coefficients().items()) == idem_P1

rational_lift_P1

1/10*() + 3/80*(3,4,5) + 3/80*(3,5,4) - 1/40*(2,3)(4,5) - 1/40*(2,3,4) - 1/40*(2,3,5) - 1/40*(2,4,3) + 3/80*(2,4,5) + 3/80*(2,4)(3,5) - 1/40*(2,5,3) + 3/80*(2,5,4) + 3/80*(2,5)(3,4) + 3/80*(1,2)(4,5) + 1/10*(1,2)(3,4) + 3/80*(1,2)(3,5) - 1/40*(1,2,3) + 3/80*(1,2,3,4,5) + 3/80*(1,2,3,5,4) - 1/40*(1,2,4,5,3) - 1/40*(1,2,4) - 1/40*(1,2,4,3,5) - 1/40*(1,2,5,4,3) + 3/80*(1,2,5) + 3/80*(1,2,5,3,4) - 1/40*(1,3,2) - 1/40*(1,3,4,5,2) - 1/40*(1,3,5,4,2) + 3/80*(1,3)(4,5) - 1/40*(1,3,4) + 3/80*(1,3,5) + 1/10*(1,3)(2,4) + 3/80*(1,3,2,4,5) + 3/80*(1,3,5,2,4) + 3/80*(1,3)(2,5) + 3/80*(1,3,2,5,4) - 1/40*(1,3,4,2,5) + 3/80*(1,4,5,3,2) - 1/40*(1,4,2) + 3/80*(1,4,3,5,2) - 1/40*(1,4,3) - 1/40*(1,4,5) - 1/40*(1,4)(3,5) + 3/80*(1,4,5,2,3) + 1/10*(1,4)(2,3) + 3/80*(1,4,2,3,5) + 3/80*(1,4,2,5,3) + 3/80*(1,4,3,2,5) - 1/40*(1,4)(2,5) + 3/80*(1,5,4,3,2) + 3/80*(1,5,2) - 1/40*(1,5,3,4,2) + 3/80*(1,5,3) - 1/40*(1,5,4) + 3/80*(1,5)(3,4) + 3/80*(1,5,4,2,3) - 1/40*(1,5)(2,3) + 3/80*(1,5,2,3,4) - 1/40*(1,5,2,4,3) + 3

In [5]:
###############
# For idem_P4 #
###############

import pandas as pd


QA = GroupAlgebra(A5,QQ)
num_coeffs_of_accuracy_in_lift_refinement = 40


initial_lift_of_idem_P4 = sum(int(coeff)*QA(g) for (g,coeff) in idem_P4.monomial_coefficients().items())
refined_lift_of_idem_P4 = refine_lift(initial_lift_of_idem_P4, num_coeffs_of_accuracy_in_lift_refinement)
refined_lift_of_idem_P4_3_adic_coefficients = int_coeffs_to_3adic(refined_lift_of_idem_P4.monomial_coefficients(), num_coeffs_of_accuracy_in_lift_refinement)

idem_P4_lift_df = pd.DataFrame.from_dict(refined_lift_of_idem_P4_3_adic_coefficients, orient='index',columns=['3-adic coefficients'])
idem_P4_lift_df['rational_coefficients'] = idem_P4_lift_df['3-adic coefficients'].apply(rational_from_3_adic)
idem_P4_lift_df.index.name = "A5 element"

rational_lift_P4 = (idem_P4_lift_df.index.to_series().apply(lambda x: QA(x)) * idem_P4_lift_df['rational_coefficients']).sum()

#is it idempotent?
assert rational_lift_P4**2 == rational_lift_P4
#does it reduce modulo 3 to idem_P4?
assert sum(GF(3)(coeff)*A(ele) for (ele,coeff) in rational_lift_P4.monomial_coefficients().items()) == idem_P4

rational_lift_P4

3/20*() - 3/80*(3,4,5) - 3/80*(3,5,4) + 1/40*(2,3)(4,5) + 1/40*(2,3,4) + 1/40*(2,3,5) + 1/40*(2,4,3) - 3/80*(2,4,5) - 3/80*(2,4)(3,5) + 1/40*(2,5,3) - 3/80*(2,5,4) - 3/80*(2,5)(3,4) - 3/80*(1,2)(4,5) + 3/20*(1,2)(3,4) - 3/80*(1,2)(3,5) + 1/40*(1,2,3) - 3/80*(1,2,3,4,5) - 3/80*(1,2,3,5,4) + 1/40*(1,2,4,5,3) + 1/40*(1,2,4) + 1/40*(1,2,4,3,5) + 1/40*(1,2,5,4,3) - 3/80*(1,2,5) - 3/80*(1,2,5,3,4) + 1/40*(1,3,2) + 1/40*(1,3,4,5,2) + 1/40*(1,3,5,4,2) - 3/80*(1,3)(4,5) + 1/40*(1,3,4) - 3/80*(1,3,5) + 3/20*(1,3)(2,4) - 3/80*(1,3,2,4,5) - 3/80*(1,3,5,2,4) - 3/80*(1,3)(2,5) - 3/80*(1,3,2,5,4) + 1/40*(1,3,4,2,5) - 3/80*(1,4,5,3,2) + 1/40*(1,4,2) - 3/80*(1,4,3,5,2) + 1/40*(1,4,3) + 1/40*(1,4,5) + 1/40*(1,4)(3,5) - 3/80*(1,4,5,2,3) + 3/20*(1,4)(2,3) - 3/80*(1,4,2,3,5) - 3/80*(1,4,2,5,3) - 3/80*(1,4,3,2,5) + 1/40*(1,4)(2,5) - 3/80*(1,5,4,3,2) - 3/80*(1,5,2) + 1/40*(1,5,3,4,2) - 3/80*(1,5,3) + 1/40*(1,5,4) - 3/80*(1,5)(3,4) - 3/80*(1,5,4,2,3) + 1/40*(1,5)(2,3) - 3/80*(1,5,2,3,4) + 1/40*(1,5,2,4,3) - 3

# A final sanity check

We now have lifts of the idempotents! That is, we have lifted idempotents from $\mathbb{F}_3A_5$ to $\mathbb{Z}_{(3)}A_5 \subset \mathbb{Q}A_5$.

As a final sanity check we compute the dimension of the vector space $\mathbb{Q}A_5e$ for each of the idempoents $e \in \mathbb{Q}A_5$.

In [6]:
def dimAe(e):
    """
    Given idempotent e, find dimension of QA_5e
    """
    spanning_set = [g * e for g in A5]

    #create coefficient matrix for each element, the rank of this matrix is the rank/dimension.
    coeff_array = []
    for ele in spanning_set:
        coeffs = ele.monomial_coefficients() #dict of form{'<grp_element>': <coeff>}
        coeff_list = []
        for g in A5:
            try:
                #if g has non-zero coefficient, coeffs[g] will return it, else it has 0 coefficient
                coeff_list.append(coeffs[g])
            except:
                coeff_list.append(0)
        coeff_array.append(coeff_list)
    coeff_matrix = Matrix(QQ, coeff_array)
    rank_of_matrix = coeff_matrix.rank()
    return rank_of_matrix

print(f"The dimension of QA_5(rational_lift_of_idem_P1) as a Q-vector space is {dimAe(rational_lift_P1)}")
print(f"The dimension of QA_5(rational_lift_of_idem_P4) as a Q-vector space is {dimAe(rational_lift_P4)}")

The dimension of QA_5(rational_lift_of_idem_P1) as a Q-vector space is 6
The dimension of QA_5(rational_lift_of_idem_P4) as a Q-vector space is 9
